In [1]:
#import shutil

#old = r'/global/u1/s/sgmoore1/ssl-legacysurvey'
#new = r'/pscratch/sd/s/sgmoore1'

#shutil.move(old, new)

In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import h5py
import math

In [2]:
desc= Table.read('/global/homes/s/sgmoore1/SGA/py/SGA/data/SGA2020/SGA2020-missing-galaxies.ecsv')

In [2]:
skies= Table.read('/pscratch/sd/s/sgmoore1/SGA2024/catalogs/skies.fits',hdu=1)
missing=Table.read('/pscratch/sd/s/sgmoore1/SGA2024/catalogs/sga2020-missing.fits',hdu=1)
sga2020 = Table.read('/pscratch/sd/s/sgmoore1/SGA2024/catalogs/sga2020.fits',hdu=1)
SGA= Table.read('/pscratch/sd/s/sgmoore1/SGA2024/catalogs/SGA2024.fits',hdu=1)
offset = Table.read('/pscratch/sd/s/sgmoore1/SGA2024/catalogs/sga2020-offset.fits',hdu=1)

In [3]:
missing

GALAXY,RA,DEC,DIAM,MAG_G,MAG_R,MAG_Z
,deg,deg,arcmin,,,
bytes30,float64,float64,float32,int64,int64,int64
2MASXJ00000256+0817537,0.0106,8.298,1.0,18,18,18
2MASXJ00000896+0817338,0.0373,8.2927,1.0,18,18,18
DR10-0001m365-1293,0.0414,-36.5683,1.0,18,18,18
DR10-0001m405-3541,0.085,-40.4098,1.0,18,18,18
PGC000026,0.115,-7.8823,1.0,18,18,18
AGC102729,0.1338,30.8696,1.0,18,18,18
PGC618004,0.1584,-37.7653,1.0,18,18,18
2MASXJ00003907-4938479,0.1627,-49.6467,1.0,18,18,18


In [5]:
# from matplotlib import pyplot as plt
# import numpy as np

# # Creating histogram
# fig, ax = plt.subplots(figsize =(10, 7))
# ax.hist(sga2020['D26'], bins=50)
# ax.vlines(x = 0.664 , ymin=0, ymax=550, color='k')

# # Show plot
# plt.show()


In [4]:
# NEEDED TO CORRECT FOR EMPTY CUTOUTS, BUT SAVED CORRECTED VERSION
# i=0
# while i < 264:
#     if (missing['RA'][i] == 0.11) or (missing['RA'][i] == 0.6646) or (missing['RA'][i] == 0.2534) or  (missing['RA'][i] == 0.8861) or (missing['RA'][i] == 0.5981) or  (missing['RA'][i] == 0.242)or (missing['RA'][i] == 0.4408) or (missing['RA'][i] == 0.6854) or (missing['RA'][i] == 0.9928 )or (missing['RA'][i] == 0.6893):
#         missing['RA'][i] = -250
#     else:
#         missing['RA'][i] = missing['RA'][i]
#     i+=1

In [6]:
# missing.sort('RA')
# missing.remove_rows(range(0,10))
# missing
# #temporary removal of the 10 cutouts that had problems pulling
# #Also had to adjust RA=0.56699 to that because it kept rounded to 0.567 and was pulling wonky

GALAXY,RA,DEC,DIAM
,deg,deg,arcmin
bytes30,float64,float64,float32
2MASXJ00000256+0817537,0.0106,8.298,1.0
2MASXJ00000896+0817338,0.0373,8.2927,1.0
DR10-0001m365-1293,0.0414,-36.5683,1.0
DR10-0001m405-3541,0.085,-40.4098,1.0
PGC000026,0.115,-7.8823,1.0
AGC102729,0.1338,30.8696,1.0
PGC618004,0.1584,-37.7653,1.0
2MASXJ00003907-4938479,0.1627,-49.6467,1.0


In [14]:
offset

SGA_ID,GALAXY,RA,DEC,D26,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,norm_shift,SHIFT
int64,bytes29,float64,float64,float32,float32,float32,float32,float64,float64
24,PGC2162969,247.4246173214296,40.250675768609874,0.5431378,-1.0,-1.0,-1.0,0.5481583832730266,0.002481046101567214
77,SDSSJ151648.69+140504.0,229.20170057806905,14.087954271624621,0.39918694,17.963633,17.444605,17.110806,1.1174140481157553,0.00371714244553448
181,PGC1400374,211.32778969179694,11.890441626492535,0.4241666,17.739927,16.70688,16.014826,0.1583330141237253,0.0005596631223448383
308,PGC2094581,120.16097945653722,37.18046059005803,0.6698354,16.886106,15.830816,15.131019,0.6420719294433813,0.0035840208367221675
351,SDSSJ102543.09+093119.8,156.43415983759374,9.517356020040454,0.57250285,18.072153,17.088245,16.241909,1.3967152434054175,0.006663528829698766
556,PGC1593038,126.03479628385293,19.454549028141486,0.6886622,16.907436,16.274525,15.739177,1.1502682965307567,0.006601218856165379
707,PGC2030436,230.16683190969047,33.369139993167146,0.72314805,16.420382,15.458088,14.904183,0.9392656536462491,0.005660234365969868
720,PGC2545412,196.3025953763888,56.70972395876638,0.4072858,16.63563,16.219702,15.992032,1.092611838144155,0.003708377474885814
740,PGC2279002,129.0032887229085,46.31126780303184,0.7394494,16.10833,15.258259,14.661092,0.7470894825864692,0.00460362380057496


In [12]:
skies

RA,DEC,DESI_TARGET,HPXPIXEL,MAG_G,MAG_R,MAG_Z,FLUX_G,FLUX_R,FLUX_Z
float64,float64,int64,int64,int64,int64,int64,int64,int64,int64
0.2920276794798619,-44.87814795867059,4294967296,35466,20,20,20,10,10,10
0.5341639203503453,-59.49849861072,4294967296,34944,20,20,20,10,10,10
0.7606003269661052,-49.21480403228082,4294967296,35368,20,20,20,10,10,10
0.7814842926587906,-50.25584512268481,4294967296,35360,20,20,20,10,10,10
1.0587581644624027,33.34086219603896,4294967296,20423,20,20,20,10,10,10
1.3817393636371509,-2.6894125040057566,4294967296,17396,20,20,20,10,10,10
2.0546265762975318,3.6771100916395585,4294967296,19474,20,20,20,10,10,10
2.5447625524523763,-0.10076063065626206,4294967296,18089,20,20,20,10,10,10
2.8502859824092304,17.69382709968802,4294967296,19872,20,20,20,10,10,10


In [39]:
# i=0
# while i < 254:
#     if skies['RA'][i] == 101.652:
#         print(i)
#     i+=1 

In [7]:
# missing.add_column(18, name='MAG_G')
# missing.add_column(18, name='MAG_R')
# missing.add_column(18, name='MAG_Z')

In [28]:
offset.add_column(0.000000 ,name='norm_shift')
offset.add_column(0.000000 ,name='SHIFT')

In [29]:
#TO CREATE OFFSET CATALOG FROM SGA-2020
import random
i=0
while i < len(offset['SGA_ID']):
    ra_shift = random.uniform(-0.005, 0.005)
    dec_shift = random.uniform(-0.005, 0.005)
    offset['RA'][i] += ra_shift
    offset['DEC'][i] += dec_shift
    offset['SHIFT'][i] = (ra_shift**2 + dec_shift**2)**0.5
    i+=1

In [32]:
i=0
while i < len(offset['SGA_ID']):
    offset['norm_shift'][i] = offset['SHIFT'][i]*120/offset['D26'][i]
    i+=1

In [11]:
offset.sort('SGA_ID')
offset

SGA_ID,GALAXY,RA,DEC,D26,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,norm_shift,SHIFT
int64,bytes29,float64,float64,float32,float32,float32,float32,float64,float64
24,PGC2162969,247.4246173214296,40.250675768609874,0.5431378,-1.0,-1.0,-1.0,0.5481583832730266,0.002481046101567214
77,SDSSJ151648.69+140504.0,229.20170057806905,14.087954271624621,0.39918694,17.963633,17.444605,17.110806,1.1174140481157553,0.00371714244553448
181,PGC1400374,211.32778969179694,11.890441626492535,0.4241666,17.739927,16.70688,16.014826,0.1583330141237253,0.0005596631223448383
308,PGC2094581,120.16097945653722,37.18046059005803,0.6698354,16.886106,15.830816,15.131019,0.6420719294433813,0.0035840208367221675
351,SDSSJ102543.09+093119.8,156.43415983759374,9.517356020040454,0.57250285,18.072153,17.088245,16.241909,1.3967152434054175,0.006663528829698766
556,PGC1593038,126.03479628385293,19.454549028141486,0.6886622,16.907436,16.274525,15.739177,1.1502682965307567,0.006601218856165379
707,PGC2030436,230.16683190969047,33.369139993167146,0.72314805,16.420382,15.458088,14.904183,0.9392656536462491,0.005660234365969868
720,PGC2545412,196.3025953763888,56.70972395876638,0.4072858,16.63563,16.219702,15.992032,1.092611838144155,0.003708377474885814
740,PGC2279002,129.0032887229085,46.31126780303184,0.7394494,16.10833,15.258259,14.661092,0.7470894825864692,0.00460362380057496


In [39]:
offset.write('/pscratch/sd/s/sgmoore1/SGA2024/catalogs/sga2020-offset.fits', overwrite=True)

In [4]:
missing

GALAXY,RA,DEC,DIAM,MAG_G,MAG_R,MAG_Z
,deg,deg,arcmin,,,
bytes30,float64,float64,float32,int64,int64,int64
2MASXJ00000256+0817537,0.0106,8.298,1.0,18,18,18
2MASXJ00000896+0817338,0.0373,8.2927,1.0,18,18,18
DR10-0001m365-1293,0.0414,-36.5683,1.0,18,18,18
DR10-0001m405-3541,0.085,-40.4098,1.0,18,18,18
PGC000026,0.115,-7.8823,1.0,18,18,18
AGC102729,0.1338,30.8696,1.0,18,18,18
PGC618004,0.1584,-37.7653,1.0,18,18,18
2MASXJ00003907-4938479,0.1627,-49.6467,1.0,18,18,18


SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

In [5]:
sga2020

SGA_ID,GALAXY,RA,DEC,D26,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26
int64,bytes29,float64,float64,float32,float32,float32,float32
1124542,PGC2284265,285.76484806273606,46.61915211315246,0.543673,16.840738,16.150396,15.776626
333794,PGC1005872,71.05058720372335,-8.271502492025583,0.7995004,16.424429,15.474097,14.7874975
977036,PGC705877,8.052927476994988,-31.377953085071276,0.5899052,18.459743,17.272573,16.518524
314144,PGC947262,315.0368779082185,-12.810440883819604,0.7997221,16.653307,15.702285,15.054771
69798,PGC866594,76.40908976066305,-18.563872537341794,0.62792826,-1.0,-1.0,-1.0
1385078,PGC2072267,213.5129676741262,35.93476138975581,0.9732259,16.199339,15.656358,15.374508
411069,PGC1052082,31.740346720769168,-4.892680204561648,0.66989154,16.458628,15.824331,15.369906
1204357,PGC2696150,189.11624478705633,67.18030742634083,0.41621464,17.249083,16.64695,16.275515
765200,PGC1255268,347.6945096847362,3.5983692137140944,0.45315737,16.633718,16.03178,15.589279


In [40]:
skies


RA,DEC,DESI_TARGET,HPXPIXEL,MAG_G,MAG_R,MAG_Z,FLUX_G,FLUX_R,FLUX_Z
float64,float64,int64,int64,int64,int64,int64,int64,int64,int64
46.29542347920933,5.817355354123844,68719476736,27,20,20,20,10,10,10
29.15345938642771,16.643333698912397,4294967296,646,20,20,20,10,10,10
123.53750231248975,35.38264479360898,4294967296,6415,20,20,20,10,10,10
129.70572107142664,44.900797970542214,4294967296,6518,20,20,20,10,10,10
120.85665365921263,41.64858028926321,4294967296,6547,20,20,20,10,10,10
119.18730724026115,46.034236351698944,4294967296,6624,20,20,20,10,10,10
100.353908930381,34.57854459808884,4294967296,6670,20,20,20,10,10,10
105.15925402912588,40.9862037896435,4294967296,6756,20,20,20,10,10,10
99.06656483573545,39.61160317823219,4294967296,6801,20,20,20,10,10,10


In [7]:
gals

GALAXY,RA,DEC,DIAM
,deg,deg,arcmin
bytes30,float64,float64,float32
GALEXASC J010318.95-044018.0,15.8297,-4.671559,0.8
UGCA020,25.811359,19.975461,1.5
PGC061664,274.039,42.6564,2.0
MCG +07-37-031,274.048125,42.660333,0.8
UGC 06073 NED02,164.94372,17.65347,0.5
ARP 148 NED02,165.969792,40.849083,0.5
PGC070787,348.643309,-43.603653,4.498
[KKH2007]J1228+4358,187.183803,43.973394,4.0


In [13]:
#Create column to organize morphologies by Hubble type
skies.add_column(20, name='MAG_G')
skies.add_column(20, name='MAG_R')
skies.add_column(20, name='MAG_Z')
skies.add_column(10, name='FLUX_G')
skies.add_column(10, name='FLUX_R')
skies.add_column(10, name='FLUX_Z')


In [50]:
skies.write('/pscratch/sd/s/sgmoore1/SGA2024/catalogs/skies.fits', overwrite=True)

In [40]:
skies

RA,DEC,DESI_TARGET,HPXPIXEL,MAG_G,MAG_R,MAG_Z,FLUX_G,FLUX_R,FLUX_Z
float64,float64,int64,int64,int64,int64,int64,int64,int64,int64
0.2920276794798619,-44.87814795867059,4294967296,35466,20,20,20,10,10,10
0.5341639203503453,-59.49849861072,4294967296,34944,20,20,20,10,10,10
0.7606003269661052,-49.21480403228082,4294967296,35368,20,20,20,10,10,10
0.7814842926587906,-50.25584512268481,4294967296,35360,20,20,20,10,10,10
1.0587581644624027,33.34086219603896,4294967296,20423,20,20,20,10,10,10
1.3817393636371509,-2.6894125040057566,4294967296,17396,20,20,20,10,10,10
2.0546265762975318,3.6771100916395585,4294967296,19474,20,20,20,10,10,10
2.5447625524523763,-0.10076063065626206,4294967296,18089,20,20,20,10,10,10
2.8502859824092304,17.69382709968802,4294967296,19872,20,20,20,10,10,10


In [15]:
missing

GALAXY,RA,DEC,DIAM,MAG_G,MAG_R,MAG_Z
,deg,deg,arcmin,,,
bytes30,float64,float64,float32,int64,int64,int64
2MASXJ00000256+0817537,0.0106,8.298,1.0,18,18,18
2MASXJ00000896+0817338,0.0373,8.2927,1.0,18,18,18
DR10-0001m365-1293,0.0414,-36.5683,1.0,18,18,18
DR10-0001m405-3541,0.085,-40.4098,1.0,18,18,18
PGC000026,0.115,-7.8823,1.0,18,18,18
AGC102729,0.1338,30.8696,1.0,18,18,18
PGC618004,0.1584,-37.7653,1.0,18,18,18
2MASXJ00003907-4938479,0.1627,-49.6467,1.0,18,18,18


In [9]:
#missing.write('/pscratch/sd/s/sgmoore1/SGA2024/catalogs/sga2020-missing.fits', overwrite=True)

In [6]:
f = h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/disk_listed_offsets.h5','a')

In [7]:
count = (5000+5000+999)
pixel = 152
#brickid = f.create_dataset('brickid',(count,), maxshape=(None,))
# flux_g = f.create_dataset('flux_g',(count,), maxshape=(None,))       #needed
# flux_r = f.create_dataset('flux_r',(count,), maxshape=(None,))       #needed
# flux_z = f.create_dataset('flux_z',(count,), maxshape=(None,))       #needed
#fiberflux_g = f.create_dataset('fiberflux_g',(count,), maxshape=(None,))
#fiberflux_r = f.create_dataset('fiberflux_r',(count,), maxshape=(None,))
#fiberflux_z = f.create_dataset('fiberflux_z',(count,), maxshape=(None,))
#psfsize_g = f.create_dataset('psfsize_g',(count,), maxshape=(None,))       #needed
#psfsize_r = f.create_dataset('psfsize_r',(count,), maxshape=(None,))       #needed
#psfsize_z = f.create_dataset('psfsize_z',(count,), maxshape=(None,))       #needed
#psfdepth_g = f.create_dataset('psfdepth_g',(count,), maxshape=(None,))       #needed
#psfdepth_r = f.create_dataset('psfdepth_r',(count,), maxshape=(None,))       #needed
#psfdepth_z = f.create_dataset('psfdepth_z',(count,), maxshape=(None,))       #needed
#nobs_g = f.create_dataset('nobs_g',(count,), maxshape=(None,))
#nobs_r = f.create_dataset('nobs_r',(count,), maxshape=(None,))
#nobs_z = f.create_dataset('nobs_z',(count,), maxshape=(None,))
# fracin_g = f.create_dataset('fracin_g',(count,), maxshape=(None,))
# fracin_r = f.create_dataset('fracin_r',(count,), maxshape=(None,))
# fracin_z = f.create_dataset('fracin_z',(count,), maxshape=(None,))
# release = f.create_dataset('release',(count,), maxshape=(None,))
# objid = f.create_dataset('objid',(count,), maxshape=(None,))
# source_type = f.create_dataset('source_type',(count,), maxshape=(None,))
# maskbits = f.create_dataset('maskbits',(count,), maxshape=(None,))
# ebv = f.create_dataset('ebv',(count,), maxshape=(None,))       #needed
images = f.create_dataset('images',(count,3,pixel,pixel), maxshape=(None, 3, pixel, pixel))       #needed
ra = f.create_dataset('ra',(count,), maxshape=(None,))       #needed
dec = f.create_dataset('dec',(count,), maxshape=(None,))       #needed
mag_g = f.create_dataset('mag_g',(count,), maxshape=(None,))
mag_r = f.create_dataset('mag_r',(count,), maxshape=(None,))
mag_z = f.create_dataset('mag_z',(count,), maxshape=(None,))
cutout_type = f.create_dataset('cutout_type',(count,), maxshape=(None,))
umap_region = f.create_dataset('umap_region',(count,), maxshape=(None,))
offsets = f.create_dataset('offset',(count,), maxshape=(None,))
#morph = f.create_dataset('morphology',(count,), maxshape=(None,))
#PROBLEMATIC ONE:
#z_phot_median = f.create_dataset('z_phot_median',(count,), maxshape=(None,))

In [47]:
# ra = np.array((4096,))
# dec = np.array((4096,))
# mag_g = np.array((4096,))
# mag_r = np.array((4096,))
# mag_z = np.array((4096,))
# count_type  = np.array((4096,))

In [8]:
#NEEDED TO CORRECT FOR A REPEATING 9 THAT CAUSES PROBLEMS WITH FILE GRABBING
#missing['RA'][42] = 0.56699

In [15]:
j=0
while j < 5000:
    f1 = math.trunc(sga2020['RA'][j])
    f2 = math.trunc((1000*sga2020['RA'][j]))
    f3 = math.trunc((100000*sga2020['RA'][j]))
    f4 = 'm' if sga2020['DEC'][j] < 0 else 'p'
    f5 = abs(math.trunc((100000*sga2020['DEC'][j])))
    
    
    img = fits.open('/pscratch/sd/s/sgmoore1/SGA2024/cutouts/{:03d}/{:06d}/{:08d}{}{:07d}.fits'.format(f1,f2,f3,f4,f5))
    
    #fill in rest of data for h5 file
    images[j] = np.array(img[0].data)
    # flux_g[j] = skies['FLUX_G'][j]
    # flux_r[j] = skies['FLUX_R'][j]
    # flux_z[j] = skies['FLUX_Z'][j]
    ra[j] = sga2020['RA'][j]
    dec[j] = sga2020['DEC'][j]
    mag_g[j] = sga2020['G_MAG_SB26'][j]
    mag_r[j] = sga2020['R_MAG_SB26'][j]
    mag_z[j] = sga2020['Z_MAG_SB26'][j]
    cutout_type[j] = 1
    offsets[j] = 0
    j+=1

In [16]:
j=5000
i=0
while i < 5000:
    f1 = math.trunc(offset['RA'][i])
    f2 = math.trunc((1000*offset['RA'][i]))
    f3 = math.trunc((100000*offset['RA'][i]))
    f4 = 'm' if offset['DEC'][i] < 0 else 'p'
    f5 = abs(math.trunc((100000*offset['DEC'][i])))
    
    
    img = fits.open('/pscratch/sd/s/sgmoore1/SGA2024/off_cutouts/{:03d}/{:06d}/{:08d}{}{:07d}.fits'.format(f1,f2,f3,f4,f5))
    
    #fill in rest of data for h5 file
    images[j] = np.array(img[0].data)
    # flux_g[j] = skies['FLUX_G'][j]
    # flux_r[j] = skies['FLUX_R'][j]
    # flux_z[j] = skies['FLUX_Z'][j]
    ra[j] = offset['RA'][i]
    dec[j] = offset['DEC'][i]
    mag_g[j] = offset['G_MAG_SB26'][i]
    mag_r[j] = offset['R_MAG_SB26'][i]
    mag_z[j] = offset['Z_MAG_SB26'][i]
    cutout_type[j] = 2
    offsets[j] = offset['norm_shift'][i]
    
    j+=1
    i+=1

In [17]:
j=(10000)
jj=0
while jj < 999:
    f1 = math.trunc(skies['RA'][jj])
    f2 = math.trunc((1000*skies['RA'][jj]))
    f3 = math.trunc((100000*skies['RA'][jj]))
    f4 = 'm' if skies['DEC'][jj] < 0 else 'p'
    f5 = abs(math.trunc((100000*skies['DEC'][jj])))
    
    
    img = fits.open('/pscratch/sd/s/sgmoore1/SGA2024/cutouts/{:03d}/{:06d}/{:08d}{}{:07d}.fits'.format(f1,f2,f3,f4,f5))
    
    #fill in rest of data for h5 file
    images[j] = np.array(img[0].data)
    # flux_g[j] = skies['FLUX_G'][j]
    # flux_r[j] = skies['FLUX_R'][j]
    # flux_z[j] = skies['FLUX_Z'][j]
    ra[j] = skies['RA'][jj]
    dec[j] = skies['DEC'][jj]
    mag_g[j] = skies['MAG_G'][jj]
    mag_r[j] = skies['MAG_R'][jj]
    mag_z[j] = skies['MAG_Z'][jj]
    cutout_type[j] = 3
    offsets[j] = 5
    j+=1
    jj+=1

In [38]:
f = h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/disk_listed_offsets.h5','a').close

In [4]:
with h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/disk_listed_offsets.h5', 'a') as hf:
    print(hf['offset'][10900])

2.7


In [20]:
# j = (10000+999)
# i=0
# while i < 254:
#     f1 = math.trunc(missing['RA'][i])
#     f2 = math.trunc((1000*missing['RA'][i]))
#     f3 = math.trunc((100000*missing['RA'][i]))
#     f4 = 'm' if missing['DEC'][i] < 0 else 'p'
#     f5 = abs(math.trunc((100000*missing['DEC'][i])))
    
    
#     img = fits.open('/pscratch/sd/s/sgmoore1/SGA2024/cutouts/{:03d}/{:06d}/{:08d}{}{:07d}.fits'.format(f1,f2,f3,f4,f5))
    
#     #fill in rest of data for h5 file
#     images[j] = np.array(img[0].data)
#     # # flux_g[i] = skies['FLUX_G'][i]
#     # # flux_r[i] = skies['FLUX_R'][i]
#     # # flux_z[i] = skies['FLUX_Z'][i]
#     ra[j] = missing['RA'][i]
#     dec[j] = missing['DEC'][i]
#     mag_g[j] = missing['MAG_G'][i]
#     mag_r[j] = missing['MAG_R'][i]
#     mag_z[j] = missing['MAG_Z'][i]
#     cutout_type[j] = 3
#     j+=1
#     i+=1

In [48]:
# j=0 
# while j < 254:
#     f1 = int(missing['RA'][j])
#     f2 = int((1000*missing['RA'][j]))
#     f3 = int((100000*missing['RA'][j]))
#     f4 = 'm' if missing['DEC'][j] < 0 else 'p'
#     f5 = int(100000*np.abs(missing['DEC'][j]))
    
    
#     img = fits.open('/pscratch/sd/s/sgmoore1/SGA2024/cutouts/{:03d}/{:06d}/{:08d}{}{:07d}.fits'.format(f1,f2,f3,f4,f5))
    
#     #fill in rest of data for h5 file
#     images[j] = np.array(img[0].data)
#     # flux_g[j] = skies['FLUX_G'][j]
#     # flux_r[j] = skies['FLUX_R'][j]
#     # flux_z[j] = skies['FLUX_Z'][j]
#     # ra[j] = sga2020['RA'][j]
#     # dec[j] = sga2020['DEC'][j]
#     # mag_g[j] = sga2020['G_MAG_SB26'][j]
#     # mag_r[j] = sga2020['R_MAG_SB26'][j]
#     # mag_z[j] = sga2020['Z_MAG_SB26'][j]
#     # cutout_type[j] = 1
#     j+=1

In [66]:
missing

GALAXY,RA,DEC,DIAM,MAG_G,MAG_R,MAG_Z
,deg,deg,arcmin,,,
bytes30,float64,float64,float32,int64,int64,int64
PGC1814250,0.1731,27.6514,1.0,18,18,18
PGC000057,0.1765,28.3691,0.28,18,18,18
PGC124799,0.1787,-39.4361,1.0,18,18,18
PGC508513,0.1935,-46.9581,1.0,18,18,18
PGC000067,0.2341,-49.5213,1.0,18,18,18
PGC783590,0.242,-25.0141,1.0,18,18,18
DR10-0001m315-6306,0.2534,-31.3779,1.0,18,18,18
PGC495312,0.2721,-47.8137,1.0,18,18,18


<HDF5 dataset "images": shape (4224, 3, 152, 152), type "<f4">

In [49]:
newsize = (4096 + 128 + 4096) + 254

with h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/faroffs.h5', 'a') as hf:
    hf['images'].resize(newsize, axis = 0)
    hf['images'][-254:] = images

    hf['mag_g'].resize(newsize, axis = 0)
    hf['mag_g'][-254:] = 18

    hf['mag_r'].resize(newsize, axis = 0)
    hf['mag_r'][-254:] = 18
    
    hf['mag_z'].resize(newsize, axis = 0)
    hf['mag_z'][-254:] = 18
    
    
#     hf['flux_g'].resize((hf['flux_g'].shape[0] + skies['FLUX_G'].shape[0]), axis = 0)
#     hf['flux_g'][-skies['FLUX_G'].shape[0]:] = skies['FLUX_G']

#     hf['flux_r'].resize((hf['flux_r'].shape[0] + skies['FLUX_R'].shape[0]), axis = 0)
#     hf['flux_r'][-skies['FLUX_R'].shape[0]:] = skies['FLUX_R']
    
#     hf['flux_z'].resize((hf['flux_z'].shape[0] + skies['FLUX_Z'].shape[0]), axis = 0)
#     hf['flux_z'][-skies['FLUX_Z'].shape[0]:] = skies['FLUX_Z']

#     hf['ebv'].resize(5628,axis=0)

    hf['ra'].resize(newsize, axis = 0)
    hf['ra'][-254:] = missing['RA']
    hf['dec'].resize(newsize, axis = 0)
    hf['dec'][-254:] = missing['DEC']
    
#     hf['brickid'].resize(newsize,axis=0)
#     hf['fiberflux_g'].resize(newsize,axis=0)
#     hf['fiberflux_r'].resize(newsize,axis=0)
#     hf['fiberflux_z'].resize(newsize,axis=0)
#     hf['fracin_g'].resize(newsize,axis=0)
#     hf['fracin_r'].resize(newsize,axis=0)
#     hf['fracin_z'].resize(newsize,axis=0)
#     hf['nobs_g'].resize(newsize,axis=0)
#     hf['nobs_r'].resize(newsize,axis=0)
#     hf['nobs_z'].resize(newsize,axis=0)
#     hf['psfdepth_g'].resize(newsize,axis=0)
#     hf['psfdepth_r'].resize(newsize,axis=0)
#     hf['psfdepth_z'].resize(newsize,axis=0)
#     hf['psfsize_g'].resize(newsize,axis=0)
#     hf['psfsize_r'].resize(newsize,axis=0)
#     hf['psfsize_z'].resize(newsize,axis=0)
#     hf['maskbits'].resize(newsize,axis=0) 
#     hf['objid'].resize(newsize,axis=0)
#     hf['release'].resize(newsize,axis=0)
#     hf['source_type'].resize(newsize,axis=0)
#     hf['z_phot_median'].resize(newsize,axis=0)
    
#     hf['morphology'].resize(newsize,axis=0)
#     hf['morphology'][-128:] = 20

    # cutout_type = hf.create_dataset('cutout_type',(newsize,), maxshape=(None,)) 
    hf['cutout_type'].resize(newsize, axis = 0)
    hf['cutout_type'][-254:] = 4

In [76]:
with h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/disk.h5', 'a') as hf:
    hf['cutout_type'].resize(4224, axis = 0)
    hf['cutout_type'][0:4096] = 1
    hf['cutout_type'][-128:] = 3

In [21]:
with h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/disk_offset+skies+gals.h5', 'a') as hf:
    off = hf.create_dataset('offset',(9191,), maxshape=(None,))
    i=0
    while i < 4096:
        hf['offset'][i] = 0

        hf['offset'][i+4096] = ((hf['ra'][i] - hf['ra'][4096+i])**2 + (hf['dec'][i] - hf['dec'][4096+i])**2)**0.5
        i+=1
        
    hf['offset'][8192:] = 0.01
    

In [3]:
# with h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/faroffs.h5', 'a') as hf:
#     i=0
#     while i < 4096:
#         hf['offset'][i] = 0
#         hf['offset'][i+4096] = ((hf['ra'][i] - hf['ra'][4096+i])**2 + (hf['dec'][i] - hf['dec'][4096+i])**2)
#         print(hf['offset'][i+4096])
#         i+=1
#     #print( hf['offset'][5000])
#     #hf['offset'][8192:8319] = 0.01
    
#     #mag_g = hf.create_dataset('offset',(8574,), maxshape=(None,))

In [10]:
print( hf['offset'][5000])

ValueError: Invalid location identifier (invalid location identifier)

In [7]:
f = h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/faroffs.h5','a').close

In [43]:
skies['MAG_Z']

20
20
20
20
20
20
20
20
20
20
20


In [87]:
missing[:][185]

GALAXY,RA,DEC,DIAM,MAG_G,MAG_R,MAG_Z
,deg,deg,arcmin,,,
bytes30,float64,float64,float32,int64,int64,int64
UGC08614,187.183803,43.973394,4.0,18,18,18


In [86]:
missing[:][186]

GALAXY,RA,DEC,DIAM,MAG_G,MAG_R,MAG_Z
,deg,deg,arcmin,,,
bytes30,float64,float64,float32,int64,int64,int64
[KKH2007]J1228+4358,187.183803,43.973394,4.0,18,18,18


In [ ]:
#THE FOLLOWING CODE WAS USED TO GRAB 1OO SMALL GALAXIES FROM THE LEGACYSURVEY VIEWER HTML

In [9]:
small_ras = [182.8797, 125.2038, 125.3536, 125.5098, 125.5553, 125.5998, 125.6920, 51.3533, 2.8784, 2.7559, 
             2.7556, 2.5267, 2.9239, 3.1751, 3.2240, 330.7580, 330.7674, 330.8968, 331.0626, 331.1307, 
             331.2130, 240.9218, 241.0043, 241.1663, 241.2771, 241.3181, 241.3472, 171.9610, 171.9701, 172.0442,
             172.5672, 172.6798, 172.6774, 172.7510, 172.7755, 172.8694, 172.8736, 172.8432, 173.0476, 353.1904,
             353.2904, 353.2647, 353.2358, 353.1608, 353.1850, 126.7882, 126.8119, 126.9087, 127.0289, 127.0416,
             127.2208, 127.2634, 127.2587, 127.4682, 127.6290, 128.4711, 128.9184, 54.0452, 54.2801, 54.3117,
             54.2893, 54.6743, 54.6125, 54.9459, 55.1109, 55.2546, 55.6254, 55.6584, 55.7089, 55.7602, 
             56.0658, 56.1750, 56.2749, 56.3925, 56.4981, 56.6040, 236.4994, 236.6720, 236.8201, 236.8657,
             236.9138, 237.0237, 237.0026, 237.0058, 236.9947, 236.9620, 236.9925, 237.0064, 126.7318, 126.7812,
             126.7670, 126.9084, 27.3559, 127.0468, 127.0005, 127.0755, 127.1877, 162.2132, 162.0866, 162.1425
            ]
small_decs = [12.2437, 19.4967, 19.3313, 19.3553, 19.3873, 19.3401, 19.3741, -16.2330, 29.0995, 29.3020, 
              29.2886, 29.2019, 29.3388, 29.5297, 29.5176, -9.7694, -9.7854, -9.7806, -9.7004, -9.6891,
              -9.6210, 20.7174, 20.6915, 20.6089, 20.6212, 20.5920, 20.6138, 13.4707, 13.4606, 13.3916, 
              13.1317, 13.1040, 13.0914, 13.1526, 13.1108, 13.0876, 13.0679, 13.0525, 13.0284, 15.9761,
              16.0269, 16.0541, 16.0491, 16.0490, 16.0285, 20.3141, 20.2132, 20.1832, 20.0283, 20.0484,
              19.9119, 19.9031, 19.8340, 19.5671, 19.4285, 19.2134, 19.2207, -50.6613, -50.5904, -50.5805,
              -50.6451, -50.5120, -50.5396, -50.4728, -50.4427, -50.4108, -50.4983, -50.5222, -50.4748, -50.5630,
              -50.7472, -50.8463, -50.7663, -50.8427, -50.7626, -50.6324, 28.0748, 27.9051, 28.0054, 28.1764,
              28.0990, 28.0719, 28.0541, 27.9935, 27.9656, 27.8552, 27.7911, 27.7020, 27.8081, 27.7575,
              27.7149, 27.3559, 27.2299, 27.2936, 27.4180, 27.4248, 27.6474, 12.4287, 12.4580, 12.4592
             ]

In [ ]:
import time
import requests
import shutil

#Command to retrieve url datas for FITS files
def get_fitscut(galid,ra,dec):
    url = 'https://www.legacysurvey.org/viewer/cutout.fits?ra={}&dec={}&%22/pix=0.25&layer=ls-dr9&size=152'.format(ra, dec)
    #print(url)
    res = requests.get(url, stream = True)
    file_name = 'LittleGalaxies/Gal{}.fits'.format(galid)
    
    if res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(res.raw, f)
        #print('Image sucessfully Downloaded: ',file_name)
    else:
        print('Image Couldn\'t be retrieved')

In [10]:
f = h5py.File('s.h5','w')
images = f.create_dataset('images',(100,3,152,152), maxshape=(None,3,152,152))
i=0
while i < 100:
    img = fits.open('LittleGalaxies/Gal{}.fits'.format(i))
    images[i] = np.array(img[0].data)
    i+=1


In [15]:
# with h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/from_html/152pxdata/cutoutdata.h5', 'a') as hf:
    
# #     hf['images'].resize(5449, axis = 0)
# #     hf['images'][-100:] = images

# #     hf['ra'].resize(5449, axis = 0)
# #     hf['dec'][-100:] = 18

# #     hf['mag_r'].resize(5449, axis = 0)
# #     hf['mag_r'][-100:] = 18
    
# #     hf['mag_z'].resize(5449, axis = 0)
# #     hf['mag_z'][-100:] = 18
    
# #     hf['mag_g'].resize(5449, axis = 0)
# #     hf['mag_g'][-100:] = 18   
    
# #    hf['umap_region'].resize(5449, axis = 0)
    
#     hf['morphology'].resize(5628, axis = 0) 
    
#     hf['ra'].resize(5628, axis = 0) 
    
#     hf['dec'].resize(5628, axis = 0)

    

In [14]:
# f=h5py.File('/pscratch/sd/s/sgmoore1/ssl-legacysurvey/data/from_html/152pxdata/cutoutdata.h5', 'a').close